<a href="https://colab.research.google.com/github/Sumanthhv123/Pine-wheel-assignment/blob/main/pinewheel_assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.7 MB/s eta 0:00:00


In [4]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [6]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [7]:
import os
import json
import langgraph
import pytest
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from ipaddress import ip_network, ip_address

In [8]:
# Define the agent class
class CyberSecurityAgent:
    def __init__(self, scope, max_retries=3):
        self.scope = scope  # Store user-defined scope (domains/IPs)
        self.task_list = []  # Stores dynamically generated tasks
        self.logs = []  # Stores execution logs
        self.max_retries = max_retries

In [9]:
    def add_task(self, task):
        self.task_list.append({"command": task, "retries": 0})
        print(f"[TASK ADDED] {task}")

    def execute_task(self, task):
        print(f"[EXECUTING] {task}")
        result = os.popen(task).read()
        self.logs.append({"task": task, "output": result})
        return result

In [10]:
    def enforce_scope(self, target):
        for allowed in self.scope:
            if allowed.startswith(".") and target.endswith(allowed):
                return True
            if "/" in allowed:
                try:
                    if ip_address(target) in ip_network(allowed, strict=False):
                        return True
                except ValueError:
                    continue
            if target == allowed:
                return True
        return False

In [11]:
    def run(self):
        while self.task_list:
            task_entry = self.task_list.pop(0)
            task = task_entry["command"]
            retries = task_entry["retries"]

            target = task.split()[-1]
            if self.enforce_scope(target):
                result = self.execute_task(task)
                print(f"[RESULT] {result[:200]}...")
            else:
                print(f"[ERROR] Task {task} is out of scope!")
                self.logs.append({"task": task, "error": "Out of scope"})
                if retries < self.max_retries:
                    task_entry["retries"] += 1
                    self.task_list.append(task_entry)

In [12]:
# Define LangGraph Workflow
class SecurityWorkflow:
    def __init__(self, agent):
        self.agent = agent
        self.graph = langgraph.Graph()
        self.graph.add_node("task_handler", self.task_handler)
        self.graph.set_entry_point("task_handler")

In [13]:
    def task_handler(self, inputs):
        if self.agent.task_list:
            task_entry = self.agent.task_list.pop(0)
            task = task_entry["command"]
            retries = task_entry["retries"]

            if self.agent.enforce_scope(task.split()[-1]):
                result = self.agent.execute_task(task)
                return {"next": "task_handler", "result": result}
            else:
                print(f"[ERROR] Task {task} is out of scope!")
                if retries < self.agent.max_retries:
                    task_entry["retries"] += 1
                    self.agent.task_list.append(task_entry)
                return {"next": "task_handler", "error": "Out of scope"}
        return {"next": None}  # Stop execution if no tasks are left

In [14]:
    def run(self):
        self.graph.run({})

In [15]:
# Unit tests
@pytest.fixture
def agent():
    return CyberSecurityAgent(scope=[".example.com", "192.168.1.0/24"])

def test_add_task(agent):
    agent.add_task("nmap -p 80 example.com")
    assert len(agent.task_list) == 1
    assert agent.task_list[0]["command"] == "nmap -p 80 example.com"

def test_enforce_scope(agent):
    assert agent.enforce_scope("sub.example.com") is True
    assert agent.enforce_scope("malicious-site.com") is False


In [16]:
def test_execute_task(agent, mocker):
    mocker.patch("os.popen", return_value=os)
    agent.add_task("echo 'test'")
    result = agent.execute_task("echo 'test'")
    assert "test" in result
    assert len(agent.logs) == 1


In [17]:
# Streamlit UI Deployment
def main():
    st.title("CyberSecurity Task Agent")
    st.sidebar.header("Scope Configuration")
    scope_input = st.sidebar.text_area("Enter allowed domains/IPs (comma-separated)", ".example.com, 192.168.1.0/24")
    scope_list = [s.strip() for s in scope_input.split(",")]
    agent = CyberSecurityAgent(scope=scope_list)


In [18]:
    st.sidebar.header("Task Management")
    task_input = st.sidebar.text_input("Enter a task (e.g., nmap -p 80 example.com)")
    if st.sidebar.button("Add Task"):
        agent.add_task(task_input)
        st.sidebar.success(f"Added task: {task_input}")

2025-02-26 13:19:49.806 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:19:49.923 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-26 13:19:49.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:19:49.925 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:19:49.926 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:19:49.927 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:19:49.927 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:19:49.928 Session state does not 

In [20]:
# Streamlit UI Deployment
def main():
    st.title("CyberSecurity Task Agent")
    st.sidebar.header("Scope Configuration")
    scope_input = st.sidebar.text_area("Enter allowed domains/IPs (comma-separated)", ".example.com, 192.168.1.0/24")
    scope_list = [s.strip() for s in scope_input.split(",")]
    cyber_security_agent = CyberSecurityAgent(scope=scope_list) # Changed variable name to avoid conflict

    st.sidebar.header("Task Management")
    task_input = st.sidebar.text_input("Enter a task (e.g., nmap -p 80 example.com)")
    if st.sidebar.button("Add Task"):
        cyber_security_agent.add_task(task_input) # Use the instance here
        st.sidebar.success(f"Added task: {task_input}")

    if st.sidebar.button("Run Tasks"):
        cyber_security_agent.run() # Use the instance here

    st.header("Execution Logs")
    for log in cyber_security_agent.logs: # Access logs from the instance
        st.write(json.dumps(log, indent=2))

In [21]:
if __name__ == "__main__":
    main()


2025-02-26 13:21:01.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.195 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.198 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:21:01.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [22]:
import streamlit as st

query_params = st.experimental_get_query_params()
print(query_params)

2025-02-26 13:27:18.217 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:18.222 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:18.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:18.226 Please replace `st.experimental_get_query_params` with `st.query_params`.

`st.experimental_get_query_params` will be removed after 2024-04-11.

Refer to our [docs page](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.query_params) for more information.


{}


In [23]:
import os
import streamlit as st

url = os.environ.get("STREAMLIT_APP_URL")
st.write(f"The current URL is: {url}")

2025-02-26 13:27:27.036 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:27.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:27.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:27.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [24]:
import streamlit as st

query_params = st.experimental_get_query_params()

if "name" in query_params:
    name = query_params["name"][0]  # Access the value of the 'name' parameter
    st.write(f"Hello, {name}!")
else:
    st.write("Please provide a 'name' parameter in the URL.")

2025-02-26 13:27:40.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:40.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:40.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:40.908 Please replace `st.experimental_get_query_params` with `st.query_params`.

`st.experimental_get_query_params` will be removed after 2024-04-11.

Refer to our [docs page](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.query_params) for more information.
2025-02-26 13:27:40.918 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:40.919 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:27:40.923 Thread 'MainThread': missing ScriptRunContext! 